In [1]:
import numpy as np

In [2]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        if not isinstance(numpy_array, np.ndarray):
            raise ValueError("Must be initialized with a Numpy array")
        self.value = numpy_array
        self.previous_tensors = None
        self.operation = None  # type of operation done to create this tensor.
        self.grad = np.full(self.value.shape, 1)
        
    def backward(self):
        self._compute_grads(self)
    
    def _compute_grads(self, tensor):
        if tensor.previous_tensors is None:
            return
        else:
            t1, t2 = tensor.previous_tensors
            if tensor.operation == 'add':
                t1.grad = tensor.grad
                t2.grad = tensor.grad
            elif tensor.operation == 'mul':
                t1.grad = tensor.grad * t2.value 
                t2.grad = tensor.grad * t1.value
            else:
                raise ValueError("Invalid operation")
            tensor.grad = None  # remove for saving memory
            self._compute_grads(t1)
            self._compute_grads(t2)
            
    
    def _add_tensor(self, other):
        """
        Tensor addition
        """
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value + other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'add'
            new_tensor.previous_tensors = [self, other]
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
    
    def _mul_tensor(self, other):
        """
        Tensor multiplication
        """
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value * other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'mul'
            new_tensor.previous_tensors = [self, other]
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
    def _convert_other_to_tensor(self, other):
        """
        Convert 'other' to Tensor with proper shape for 
        proper operation.
        """
        # convert other to Tensor
        if isinstance(other, int) or isinstance(other, float):
            t = Tensor(np.full(self.value.shape, other))
        elif isinstance(other, np.ndarray):
            other = np.broadcast_to(other, self.value.shape)
            t = Tensor(other)
        elif isinstance(other, Tensor):
            t = other  # no need to do anything
        else:
            raise ValueError("Invalid type")
        return t
    
    def __add__(self, other):
        # convert other to tensor
        t = self._convert_other_to_tensor(other)
        # do tensor addition
        return self._add_tensor(t)
        
    def __mul__(self, other):
        # convert other to tensor
        t = self._convert_other_to_tensor(other)
        # do tensor multiplication
        return self._mul_tensor(t)
    
    def __str__(self):
        return self.value.__repr__()

In [8]:
arr1 = np.random.rand(2, 3)
arr2 = np.random.rand(2, 3)

In [9]:
t1 = Tensor(arr1)
t2 = Tensor(arr2)
print('t1:', t1)
print('t2:', t2)

t1: array([[0.79110658, 0.36495037, 0.53008584],
       [0.67279202, 0.72277842, 0.69253816]])
t2: array([[0.14182917, 0.41216405, 0.94325528],
       [0.1271352 , 0.71546317, 0.15307051]])


In [12]:
t3 = (t1 + np.full((2, 3), 1)) * np.random.rand(2, 3)

In [13]:
t3.backward()

In [14]:
t1.grad

array([[2, 2, 2],
       [2, 2, 2]])

In [5]:
t3 = Tensor(np.full(arr1.shape, 10)) * t1 * Tensor(np.full(arr1.shape, 5)) * Tensor(np.full(arr1.shape, 5)) + t2 * Tensor(np.full(arr1.shape, 3))
t4 = (t3 + Tensor(np.full(arr1.shape, 2))) * Tensor(np.full(arr1.shape, 7))

In [6]:
t4.backward()

In [7]:
t1.grad

array([[1750, 1750, 1750],
       [1750, 1750, 1750]])

In [8]:
t2.grad

array([[21, 21, 21],
       [21, 21, 21]])